# Probability 2 (Master Data science, University of Lille) / CMF (Centrale Lille, G3 SDIA)

---

## Lab 2 - Queues, Metropolis-Hastings and simulated annealing

---

## Student 1 : Hadrien Salem
## Student 2 : Emilie Salem

---

## Guidelines (read carefully before starting)

**Objectives**: numerically simulate queues, implement an example of a Metropolis-Hasting algorithm and simulated annealing.

**Setup**: after retrieving the resources for the lab on moodle:
- place the .zip archive in a local folder (Computer -> Documents/Python/);
- unzip the archive .zip;
- rename the folder with the convention lab2_Name1_Name2
- duplicate the notebook file and rename it lab2_Name1_Name2.ipynb;
- [**optional, possibly needed if working from Centrale's machines**]
    - create a `lab2` conda environment from the provided `requirement.txt` file
    ```bash
    conda create --name=lab2 --file=requirement.txt
    conda activate lab2
    # do not forget to deactivate the environment if needed
    # you can remove the environment once you are done
    conda env remove --name=lab2
    ```
    - launch jupyter notebook (the python environment will be the one from the conda environment `lab2`)
- at the end of the session, do not forget to transfer your work to your own network space if you are working on a machine from the school (do not leave your work on the C: drive).

**Assessment** &#8594; grade /20 (possibly converted later on to a grade ranging from F to A (A+))

This lab session will be evaluated, based on your answer to the exercises reported in a Jupyter notebook (e.g., this one) and any additional `.py` file produced. In particular:

- make sure the notebook you produce is properly annotated: each exercise should be introduced by a short sentence summarizing its context. Concisely answer each question from the guideline. 
- **relate the numerical results to the theory covered during the lecture** whenever appropriate;
- **codes without any explanations (in a text cell) will not be considered as a complete answer**, and will only be attributed a fraction of the grade attributed to the question.
- any code produced should be commented whenever appropriate;
- include appropriate axis labels and a relevant title to each figure reported in the notebook;
- **document any function you introduce (using docstrings)**, and comment your code whenever appropriate (see, *e.g.*, [PEP 8 style recommendations](https://www.python.org/dev/peps/pep-0008/)). 
     - use a reference docstring style, *e.g.*, the [google docstring style](https://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_google.html).
- **give a clear structure to your notebook**: longer theoretical explanations should be reported in markdown cells. Include LaTeX equations to improve the clarity of the document.

**Additional evaluation criteria**:
- Code clarity / conciseness / documentation
- Comment quality / overall presentation

## <a name="content">Contents</a>
- [Exercise 1: Simulating $M/M/1/\infty$ and $M/M/1/K$ queues](attachment:./#ex1)
- [Exercise 2: Drawing samples from the Ising model with the Metropolis-Hasings algorithm](attachment:./#ex2)
- [Exercise 3: Simulated annealing for the *traveling salesman* problem](attachment:./#ex3)

---
## <a name="ex1">Exercise 1: Simulating $M/M/1/\infty$ and $M/M/1/K$ queues</a> [(&#8593;)](attachment:./#content)

This exercise will focus on simulating an $M/M/1/\infty$ and an $M/M/1/K$ queue to illustrate some of the results covered in lecture 5. <!--in Chapter 2 and in TD.3-->

In the following, the parameter of the exponential distributions governing the inter-arrival and service times will be denoted $\lambda$ and $\mu$ respectively, with $0< \lambda < \mu $. The time instants at which changes occur in the process $X(t)$ will be denoted by $T_n$ for $n \in \mathbb{N}$. We further define:

\begin{equation}
    \rho = \frac{\lambda}{\mu}, \; X_n = X(T_n).
\end{equation}

### $M/M/1/\infty$-queue

1\. a) Using the lecture notes, implement a function to simulate a trajectory $X(t)$ of an $M/M/1/\infty$ process, where $X(t)$ represents the number of customers in the system at time $t$. <!--  from Chapter 2, Section 2.3.1 -->

> *Remark*: the signature of the function could be of the form:
``` python
def run_mm1inf(lambd: float, mu: float, rng, niter: int):
    """
    Args:
        lambd (float): birth rate.
        mu (float): death rate
        rng (int): random seed (or random generator).
        niter (int): number of changes (events) in 
                     the process.
    Raises:
        ValueError: error triggered if lambd <= 0.
        ValueError: error triggered if mu <= 0.
    Returns:
        X (array_like): trajectory (X(t_n)).
        T (array_like): time instants at which a change in 
                        the process occurs (t_n).
    """ 
```

Let us implement the function as it is described above.

We are modelling a birth/death process, and we will use the corresponding theoretical results.
We need to proceed incrementally, where at each step we compute:
- The time at which an event (birth or death) will occur.
    - If there is one or more customer in the queue, it is given by a random variable following an **exponential law** of parameter $\lambda + \mu$.
    - If there is no customer in the queue, no "death" can occur, so the time is given by a random variable following an **exponential law** of parameter $\lambda$.
- The probability that this event is a birth is given by:
    -  $\frac{\lambda}{\lambda + \mu}$ if there is one or more customer in the system.
    -  $1$ if there is no customer in the queue.

In [130]:
import numpy as np

def run_mm1inf(lambd, mu, rng, niter):
    """
    Args:
        lambd (float): birth rate.
        mu (float): death rate
        rng (int): random seed (or random generator).
        niter (int): number of changes (events) in 
                     the process.
    Raises:
        ValueError: error triggered if lambd <= 0.
        ValueError: error triggered if mu <= 0.
    Returns:
        X (array_like): trajectory (X(t_n)).
        T (array_like): time instants at which a change in 
                        the process occurs (t_n).
    """ 

    # Raises errors if mu and lambd are not valid
    if lambd < 0 or mu < 0: raise ValueError("Lambda and mu must be strictly positive.")

    # Arrays that will be returned
    X = np.zeros(niter)
    T = np.zeros(niter)

    # Initialisation
    random = np.random.default_rng(seed=rng)
    X[0] = 0
    T[0] = 0

    for i in range(1,niter) :
        # If there is no customer, birth in necessarily selected
        if X[i-1] == 0:
            event_time_increment = random.exponential(1 / lambd)
            T[i] = T[i-1] + event_time_increment
            X[i] = X[i-1] + 1

        else :
            # Generating the time increment according to an exponential law
            event_time_increment = random.exponential(1/ (lambd + mu))
            T[i] = T[i-1] + event_time_increment

            # Selection of birth or death
            birth_probability = lambd/(lambd+mu)
            r = random.binomial(1, birth_probability)
            if r == 1 : X[i] = X[i-1] + 1
            else : X[i] = X[i-1] - 1

    return X,T

b) Display the generated trajectory of the process for $(\lambda,\mu) = (5, 6)$ (e.g., use only the last 100 samples of the trajectory to obtain a representative illustration).

> *Hint*: use the function `step` from the matlplotlib library.

In [131]:
import matplotlib.pyplot as plt

# Initializing parameters
niter = 1000
lambd, mu = 5, 6
rng = 0

# Generating the trajectory
X,T = run_mm1inf(lambd, mu, rng, niter)

# Displaying the last 100 samples
plt.figure(figsize=(20,10))
plt.step(T[-100:],X[-100:])
plt.xlabel("Time at which a change in the process happens")
plt.ylabel("Number of customers")
plt.title(f"Last 100 samples of a trajectory of {niter} steps with parameters (lambda, mu) = ({lambd},{mu})")

## Comment

We can observe that the number of customers remains close to 0 throughout the simulation. That's because we chose a birth rate that is lower than the death rate: since customers leave the queue faster than they arrive, the number of people waiting in the queue rarely goes up.

2\. a) Display on the same graph the normalized histogram of $(X_n)_n$ and the stationary distribution $\pi$ defined by (see Proposition 2.1 in lecture 5)

\begin{equation}
    (\forall i \in \mathbb{N}), \; \pi(i) = (1 - \rho)\rho^i.
\end{equation}

What does this graph illustrate?

In [132]:
# Computing the stationary distribution pi
rho = lambd / mu
pi_i = lambda i: (1-rho)*(rho**i) 
lin = np.arange(0, 25)
pi_distrib = pi_i(lin)

print(pi_distrib.shape)

plt.figure(figsize=(20,10))

# Displaying the normalized histogram of X_n
X_distrib = np.bincount(X.astype(int))
plt.stem(X_distrib/niter)

# Displaying pi
plt.stem(pi_distrib, linefmt="green", markerfmt="go")

plt.title("Normalized histogram of X_n and stationary distribution pi")
plt.legend(["Normalized histogram of X_n","Stationary distribution pi"])

## Comment

We can observe that the empirical curve obtained with the simulation is close to the theoretical distribution, as expected. This is also further illustration that the number of customers tends to stay close to 0, since both graphs show it is the most visited state. 

b) Empirically evaluate the average number of customers, whose value should be close close to the theoretical value $\frac{\rho}{1-\rho}$ (see Definition 2.3 p. 2 in lecture 5).

In [133]:
# Computing the average number of customers and the theoretical value
avg_customers = X.mean()
theo_value = rho / (1-rho)

# Comparing it to the theoretical value
print(f"Average number of customers for this simulation: {avg_customers}")
print(f"Theoretical value: {theo_value}")

# Comment

As expected, the average number of customers is close to the theoretical value of 5. Again, we can note that this low value is coherent with the problem at hand.

### $M/M/1/K$

Consider the case where the size of the capacity service $K$ is finite, i.e., where the system can only accommodate up to $K$ customers. In comparison with the previous case, any new customer trying to enter the system at a time instant $t$ will be systematically rejected if $X(t) = K$.

3\. a) Propose a variant of the function developed in 1.a) to simulate a trajectory of an $M/M/1/K$ process.

The function is very similar to the one developed for the $M/M/1/\infty$ queue. The only change is that, in the event of a birth, we need to check if the maximum capacity $K$ has been reached. In that case, the number of customers remains the same since the incoming customer is rejected.

In [134]:
def run_mm1k(lambd, mu, rng, niter, K):
    """
    Args:
        lambd (float): birth rate.
        mu (float): death rate
        rng (int): random seed (or random generator).
        niter (int): number of changes (events) in 
                     the process.
        K (int): capacity service of the queue.
    Raises:
        ValueError: error triggered if lambd <= 0.
        ValueError: error triggered if mu <= 0.
    Returns:
        X (array_like): trajectory (X(t_n)).
        T (array_like): time instants at which a change in 
                        the process occurs (t_n).
    """ 

    # Raises errors if mu and lambd are not valid
    if lambd < 0 or mu < 0: raise ValueError("Lambda and mu must be strictly positive.")

    # Arrays that will be returned
    X = np.zeros(niter)
    T = np.zeros(niter)

    # Initialisation
    random = np.random.default_rng(seed=rng)
    X[0] = 0
    T[0] = 0

    for i in range(1,niter) :
        # If there is no customer, birth in necessarily selected
        if X[i-1] == 0:
            event_time_increment = random.exponential(1 / lambd)
            T[i] = T[i-1] + event_time_increment
            X[i] = X[i-1] + 1

        else :
            # Generating the time increment according to an exponential law
            event_time_increment = random.exponential(1/ (lambd + mu))
            T[i] = T[i-1] + event_time_increment

            # Selection of birth or death
            birth_probability = lambd/(lambd+mu)
            r = random.binomial(1, birth_probability)
            if r == 1 : 
                # Checking if the queue is saturated
                if X[i-1] == K:
                    X[i] = X[i-1]
                else :
                    X[i] = X[i-1] + 1
            else : X[i] = X[i-1] - 1

    return X,T

b) Display the trajectory of the process for $(\lambda,\mu, K) = (5, 6, 3)$.

In [135]:
# Initializing parameters
niter = 1000
lambd, mu, K = 5, 6, 3
rng = 0

# Generating the trajectory
X,T = run_mm1k(lambd, mu, rng, niter, K)

# Displaying the last 100 samples
plt.figure(figsize=(20,10))
plt.step(T[-100:],X[-100:])
plt.xlabel("Time at which a change in the process happens")
plt.ylabel("Number of customers")
plt.title(f"Last 100 samples of a trajectory of {niter} steps with parameters (lambda, mu, K) = ({lambd},{mu},{K})")

## Comment
We see that the number of customers never exceeds 3, which is the expected behaviour of our simulation algorithm.
Just like before, there seems to be a tendency to keep a "low" number of customers because the birth rate is lower than the death rate. This observation will be confirmed in the following segment.

4\. a) Display on the same graph the normalized histogram of $X_n$ and the stationary distribution $\pi$ defined as

$$
    (\forall i \in \mathbb{N}), \; \pi(i) =
    \begin{cases}
        \frac{(1-\rho)\rho^i}{1-\rho^{K+1}} & \text{if } i \in \{0, 1, \dotsc, K \} \\
        0 & \text{otherwise.}
    \end{cases}
$$

What does this graph illustrate?

In [138]:
# Computing the stationary distribution pi
rho = lambd / mu

pi_mm1k_i = lambda i: 0 if i not in range(0, K+1) else (1-rho)*(rho**i) / (1 - rho**(K+1))

lin = np.arange(0, 4)
pi_distrib = np.array([pi_mm1k_i(i) for i in lin]) 
    
print(pi_distrib.shape)

plt.figure(figsize=(20,10))

# Displaying the normalized histogram of X_n
X_distrib = np.bincount(X.astype(int))

plt.stem(X_distrib/niter)

# Displaying pi
plt.stem(pi_distrib, linefmt="green", markerfmt="go")

plt.title("Normalized histogram of X_n and stationary distribution pi")
plt.title("Normalized histogram of X_n and stationary distribution pi")
plt.legend(["Normalized histogram of X_n","Stationary distribution pi"])

b) Compute the theoretical average number of customers, and estimate its value using the function developed in 3.a). <!--(see TD3, exercise 3)-->

To compute the theoretical average number of customers we use the following formula:
$$
\mathbb E_{\pi}(X_n) = \sum_{i=0}^{K} i \lim_{n \rightarrow \infty} \mathbb P(X_n = i) = \sum_{i=0}^{K} i \frac{(1-\rho)\rho^i}{1-\rho^{K+1}}
$$

In [139]:
# Computing the average number of customers and the theoretical value
avg_customers = X.mean()
theo_value = ((1-rho)/(1-rho**(K+1))) * sum([n*rho**n for n in range(K+1)])

# Comparing it to the theoretical value
print(f"Average number of customers for this simulation: {avg_customers}")
print(f"Theoretical value: {theo_value}")

---
## <a name="ex2">Exercise 2: Drawing samples from the Ising model with the Metropolis-Hasings algorithm</a> [(&#8593;)](attachment:./#content)

Consider the 2D Ising model covered in Chapter 3, section 3 of the lecture notes, taken over an $N \times N$ grid for $N = 64$, with $\beta = 0.6$. <!-- Chapter 3, p.36 -->

1\. Implement a Metropolis-Hastings algorithm to draw samples from the 2D Ising model in the above configuration. Progressively display the evolution of the image as the algorithm evolves. Generate 1000 such variables, starting from a random configuration. Qualitatively comment the content of the last few generated samples, in comparison with the initial state.

> *Note*: a) the matlpotlib examples given [here](http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-as-interactive-javascript-widgets/) can be useful
>
> b) given the relatively slow evolution from one itearition to another, you may display the current state of the image only once every 20 iterations.

In [137]:
# Implementation of the Metropolis-Hastings algorithm

def metropolis_hastings():
    """
    """
    # TODO Docstring

    # Choose a symetric candidate Q
    # Compute alpha
    # Initialiser X_n à un état donné
    # A chaque itération
        # Choisir aléatoirement un état sans lequel passer suivant Q
        # Retenir ou non cet état suivant le taux d'acceptation alpha
    ...

2\. Decrease the value of the parameter $\beta = 0.1$, and generate 1000 new variables with the algorithm implemented in 1. What is the influence of the parameter $\beta$?

---
## <a name="ex3">Exercise 3: Simulated annealing for the *traveling salesman* problem</a> [(&#8593;)](attachment:./#content)

This exercise will be focused on the implementation of a basic simulated annealing algorithm to minimize a function $f : E \subset \mathbb{N} \rightarrow \mathbb{R}$, where $E$ is finite. An application to the *traveling salesman* problem will then be considered to test the algorithm implementation. 

In general, a simulated annealing algorithm can be described as follows.

---
**Simulated annealing**

Set $x_0 \in E$, $T_0  > 0$. 

$n \leftarrow 0$

While $(n \leq N)$ and $(T_n > T_{\min})$

  1. Draw a point $y \sim Q(x_n, \cdot)$ in the neighborhood of $x_n$, and $u \sim \mathscr{U}([0,1])$ (where $\mathscr{U}$ is the uniform distribution)
  
  2. Compute the acceptance probability: 
  
$$
    p(f(x_n), f(y), T_n) = 
    \begin{cases}
        1 & \text{if } f(y) < f(x_n) \\
        \text{e}^{-(f(y) - f(x_n))/T_n} & \text{otherwise.}
    \end{cases}
$$
  
  3. Set $x_{n+1} = \begin{cases}
       y & \text{if } u \leq p(f(x_n), f(y), T_n) \\
       x_n & \text{otherwise.}
       \end{cases}$
  4. Set $T_{n+1} = \frac{T_0}{\log(n+2)}$
  5. $n \leftarrow n+1$

Return $x_N$, $\bigl( f(x_n) \bigr)_{1 \leq n \leq N}$

> *Note*: in practice, the transition kernel $Q$, the neighborhood of the current point $x_n$ needs to be defined by the user, depending on the problem of interest. The definition of the other elements will be specified later for the *traveling salesman* problem.

---

1\. Implement a generic `simulated_annealing` function to be run for a maximum of $N$ iterations.

> *Note*: you can for instance create an object gathering several abstract methods to be instanciated (e.g., `draw_neighbour`, `acceptance_probability`, ...). Using functions and lambda functions is another possibility.

The algorithm implemented in 1. will be applied to the *traveling salesman* problem, briefly described in the following lines.

---
**The Traveling Salesman problem**

A salesman must find the shortest route going only once through $K$ cities, represented by the points $C_1, \dotsc, C_K$ in $\mathbb{R}^2$. In this case, the set of all possible routes $E$ contains $K!$ elements, which excludes an exhaustive search as $K$ increases (*e.g.*, for $K \geq 10$). For this application, the problem thus consists in finding a route, i.e., a permutation $\sigma$ in the symmetric group $\Sigma_K$, minimizing the function

\begin{equation}
    \sigma = \bigl( \sigma(1), \dotsc , \sigma(K) \bigr) \mapsto f(x) = \sum_{i = 1}^{K} \text{dist} (C_{\sigma(i)}, C_{\sigma(i+1)})
\end{equation}

with the convention $\sigma(K+1) = \sigma(1)$. In this case, simulated annealing randomly explores $\Sigma_K$ from one possible route to another located in its vicinity (which needs to be defined). 

In the following, the route $\tilde{\sigma} = \bigl( \tilde{\sigma}(1), \dotsc, \tilde{\sigma}(K) \bigr)$ will be said to be a neighbour of $\sigma = \bigl( \sigma(1), \dotsc, \sigma(K) \bigr)$ if there exists $1 \leq i < k \leq K $ such that:

\begin{equation}
   \bigl( \tilde{\sigma}(1), \dotsc ,\tilde{\sigma}(K) \bigr) = \bigl( \sigma(1), \dotsc, \sigma(i−1), \sigma(k), \sigma(k−1), \dotsc, \sigma(i+1), \sigma(i), \sigma(k+1), \dotsc, \sigma(K) \bigr). 
\end{equation}

For instance, for $K = 8$, the permutations $(1,2,3,4,8,7,6,5)$ and $(1,2,7,8,4,3,6,5)$ are neighbours (with $i = 3$, $k = 6$).

---

2\. Fully instantiate the simulated annealing algorithm for the traveling salesman problem. Take the $\ell_2$ norm for $\text{dist}$.

3\. Test your algorithm for $K = 50$ randomly located cities (i.e., generate the values of $(C_k)_{1 \leq k \leq K}$ randomly). Display both the evolution of the cost function with the iterations and the final route. Empirically tune the value $T_0$ to improve the result of the algorithm.

> *Notes*:
>
> a) for reproducibility, set the random seed of the random number generator to a specific value, *e.g.*, 0;
>
> b) to generate an initial path $x_0$, you can for instance randomly select a starting city $k$, and define the next city sequentially by taking its *nearest neighbour*;
>
> c) to display the trajectory, you can use `matplotlib.pyplot.arrow` to display an arrow between two consecutive points.